In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

### NASA Mars News

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [/Users/stella/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


In [3]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [4]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Retrieve the latest news
results = soup.find_all('ul', class_='item_list')
# loop over results to get article data
for result in results:
    news_title = result.find('div', class_='content_title').text
    news_p = result.find('div', class_= 'article_teaser_body').text
    print(news_title)
    print(news_p)

NASA's Curiosity Takes Selfie With 'Mary Anning' on the Red Planet
The Mars rover has drilled three samples of rock in this clay-enriched region since arriving in July.


In [6]:
# close the browser
browser.quit()

### JPL Mars Space Images - Featured Image

In [7]:
# Import BeautifulSoup
from bs4 import BeautifulSoup
from splinter import Browser

In [8]:
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [9]:
# Visit the following URL
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars#submit"
browser.visit(url)

In [10]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [11]:
results = soup.find_all('img', class_='thumb')
results[18]['src']

'/spaceimages/images/wallpaper/PIA24172-640x350.jpg'

In [12]:
# Manually copy and paste the figure link @"@
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24172-640x350.jpg'

In [13]:
# close the browser
browser.quit()

### Mars Facts

In [14]:
import pandas as pd

In [15]:
url = 'https://space-facts.com/mars/'

In [16]:
tables = pd.read_html(url)

In [17]:
mars_df = tables[2]

In [18]:
mars_df.columns = ['Description', 'Mars']
mars_df

,Description,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [19]:
mars_df.set_index('Description', inplace=True)

In [20]:
mars_df

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [21]:
html_table = mars_df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n

In [22]:
html_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

### Mars Hemispheres

In [23]:
# Import BeautifulSoup
from bs4 import BeautifulSoup
from splinter import Browser

In [24]:
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [25]:
# Visit the following URL
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [26]:
browser.find_by_text('Cerberus Hemisphere Enhanced').click()

In [27]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [28]:
fig_1 = soup.find_all('li')[0].a["href"]
fig_1

'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'

In [29]:
title_1 = soup.find('h2', class_="title").text
title_1

'Cerberus Hemisphere Enhanced'

In [30]:
browser.quit()

In [31]:
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [32]:
# Visit the following URL
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [33]:
browser.find_by_text('Schiaparelli Hemisphere Enhanced').click()

In [34]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [35]:
fig_2 = soup.find_all('li')[0].a["href"]
fig_2

'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'

In [36]:
title_2 = soup.find('h2', class_="title").text
title_2

'Schiaparelli Hemisphere Enhanced'

In [37]:
browser.quit()

In [38]:
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [39]:
# Visit the following URL
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [40]:
browser.find_by_text('Syrtis Major Hemisphere Enhanced').click()

In [41]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [42]:
fig_3 = soup.find_all('li')[0].a["href"]
fig_3

'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'

In [43]:
title_3 = soup.find('h2', class_="title").text
title_3

'Syrtis Major Hemisphere Enhanced'

In [44]:
browser.quit()

In [45]:
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [46]:
# Visit the following URL
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [47]:
browser.find_by_text('Valles Marineris Hemisphere Enhanced').click()

In [48]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [49]:
fig_4 = soup.find_all('li')[0].a["href"]
fig_4

'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'

In [50]:
title_4 = soup.find('h2', class_="title").text
title_4

'Valles Marineris Hemisphere Enhanced'

In [51]:
browser.quit()

In [52]:
hemisphere_image_urls = [
    {"title": title_1, "img_url": fig_1},
    {"title": title_2, "img_url": fig_2},
    {"title": title_3, "img_url": fig_3},
    {"title": title_4, "img_url": fig_4},
]
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]